In [10]:
import pandas as pd


In [58]:
import requests

resp = requests.get('https://exoplanetarchive.ipac.caltech.edu/cgi-bin/nstedAPI/nph-nstedAPI?table=exoplanets&format=json')
txt = resp.json()
df = pd.DataFrame(txt)
df

,pl_hostname,pl_letter,pl_name,pl_discmethod,pl_controvflag,pl_pnum,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,...,st_masslim,st_massn,st_rad,st_raderr1,st_raderr2,st_radlim,st_radn,pl_nnotes,rowupdate,pl_facility
0,Kepler-135,b,Kepler-135 b,Transit,0,2,6.002530,0.000016,-0.000016,0.0,...,NaN,1,1.27,0.24,-0.24,0.0,6,1,2014-05-14,Kepler
1,Kepler-135,c,Kepler-135 c,Transit,0,2,11.448708,0.000145,-0.000145,0.0,...,NaN,1,1.27,0.24,-0.24,0.0,6,1,2014-05-14,Kepler
2,Kepler-133,c,Kepler-133 c,Transit,0,2,31.517586,0.000158,-0.000158,0.0,...,NaN,1,1.43,0.24,-0.24,0.0,6,1,2014-05-14,Kepler
3,Kepler-134,b,Kepler-134 b,Transit,0,2,5.317429,0.000008,-0.000008,0.0,...,NaN,1,1.18,0.23,-0.23,0.0,5,1,2014-05-14,Kepler
4,Kepler-134,c,Kepler-134 c,Transit,0,2,10.105785,0.000053,-0.000053,0.0,...,NaN,1,1.18,0.23,-0.23,0.0,5,1,2014-05-14,Kepler
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4291,Kepler-1666,c,Kepler-1666 c,Transit,0,2,40.715700,NaN,NaN,0.0,...,0.0,0,1.21,0.03,-0.09,0.0,3,0,2020-09-03,Kepler
4292,Kepler-783,c,Kepler-783 c,Transit,0,2,7.053950,NaN,NaN,0.0,...,0.0,1,0.89,0.04,-0.03,0.0,5,0,2020-09-03,Kepler
4293,Kepler-1667,b,Kepler-1667 b,Transit,0,1,83.578100,NaN,NaN,0.0,...,0.0,0,1.06,0.08,-0.17,0.0,3,0,2020-09-03,Kepler
4294,Kepler-1668,b,Kepler-1668 b,Transit,0,1,15.434000,NaN,NaN,0.0,...,0.0,1,1.05,0.01,-0.01,0.0,3,0,2020-09-03,Kepler


In [59]:
df.drop(columns =['pl_letter', 'pl_discmethod'])

,pl_hostname,pl_name,pl_controvflag,pl_pnum,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbpern,pl_orbsmax,...,st_masslim,st_massn,st_rad,st_raderr1,st_raderr2,st_radlim,st_radn,pl_nnotes,rowupdate,pl_facility
0,Kepler-135,Kepler-135 b,0,2,6.002530,0.000016,-0.000016,0.0,3,0.067,...,NaN,1,1.27,0.24,-0.24,0.0,6,1,2014-05-14,Kepler
1,Kepler-135,Kepler-135 c,0,2,11.448708,0.000145,-0.000145,0.0,3,0.103,...,NaN,1,1.27,0.24,-0.24,0.0,6,1,2014-05-14,Kepler
2,Kepler-133,Kepler-133 c,0,2,31.517586,0.000158,-0.000158,0.0,4,0.204,...,NaN,1,1.43,0.24,-0.24,0.0,6,1,2014-05-14,Kepler
3,Kepler-134,Kepler-134 b,0,2,5.317429,0.000008,-0.000008,0.0,4,0.060,...,NaN,1,1.18,0.23,-0.23,0.0,5,1,2014-05-14,Kepler
4,Kepler-134,Kepler-134 c,0,2,10.105785,0.000053,-0.000053,0.0,4,0.092,...,NaN,1,1.18,0.23,-0.23,0.0,5,1,2014-05-14,Kepler
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4291,Kepler-1666,Kepler-1666 c,0,2,40.715700,NaN,NaN,0.0,2,NaN,...,0.0,0,1.21,0.03,-0.09,0.0,3,0,2020-09-03,Kepler
4292,Kepler-783,Kepler-783 c,0,2,7.053950,NaN,NaN,0.0,2,NaN,...,0.0,1,0.89,0.04,-0.03,0.0,5,0,2020-09-03,Kepler
4293,Kepler-1667,Kepler-1667 b,0,1,83.578100,NaN,NaN,0.0,2,NaN,...,0.0,0,1.06,0.08,-0.17,0.0,3,0,2020-09-03,Kepler
4294,Kepler-1668,Kepler-1668 b,0,1,15.434000,NaN,NaN,0.0,2,NaN,...,0.0,1,1.05,0.01,-0.01,0.0,3,0,2020-09-03,Kepler


In [60]:
df

,pl_hostname,pl_letter,pl_name,pl_discmethod,pl_controvflag,pl_pnum,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,...,st_masslim,st_massn,st_rad,st_raderr1,st_raderr2,st_radlim,st_radn,pl_nnotes,rowupdate,pl_facility
0,Kepler-135,b,Kepler-135 b,Transit,0,2,6.002530,0.000016,-0.000016,0.0,...,NaN,1,1.27,0.24,-0.24,0.0,6,1,2014-05-14,Kepler
1,Kepler-135,c,Kepler-135 c,Transit,0,2,11.448708,0.000145,-0.000145,0.0,...,NaN,1,1.27,0.24,-0.24,0.0,6,1,2014-05-14,Kepler
2,Kepler-133,c,Kepler-133 c,Transit,0,2,31.517586,0.000158,-0.000158,0.0,...,NaN,1,1.43,0.24,-0.24,0.0,6,1,2014-05-14,Kepler
3,Kepler-134,b,Kepler-134 b,Transit,0,2,5.317429,0.000008,-0.000008,0.0,...,NaN,1,1.18,0.23,-0.23,0.0,5,1,2014-05-14,Kepler
4,Kepler-134,c,Kepler-134 c,Transit,0,2,10.105785,0.000053,-0.000053,0.0,...,NaN,1,1.18,0.23,-0.23,0.0,5,1,2014-05-14,Kepler
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4291,Kepler-1666,c,Kepler-1666 c,Transit,0,2,40.715700,NaN,NaN,0.0,...,0.0,0,1.21,0.03,-0.09,0.0,3,0,2020-09-03,Kepler
4292,Kepler-783,c,Kepler-783 c,Transit,0,2,7.053950,NaN,NaN,0.0,...,0.0,1,0.89,0.04,-0.03,0.0,5,0,2020-09-03,Kepler
4293,Kepler-1667,b,Kepler-1667 b,Transit,0,1,83.578100,NaN,NaN,0.0,...,0.0,0,1.06,0.08,-0.17,0.0,3,0,2020-09-03,Kepler
4294,Kepler-1668,b,Kepler-1668 b,Transit,0,1,15.434000,NaN,NaN,0.0,...,0.0,1,1.05,0.01,-0.01,0.0,3,0,2020-09-03,Kepler
